base de dados:
- dados dos posts wordpress (pre-migração)
- dados dos posts wix (pós-migração)

Como organizar os dados de forma que facilite a migração e tratamento entre as duas plataformas?

- usar duckdb para compilar os dados relacionado a cada post
- importar dados brutos do wordpress pelo csv
- importar dados atuais do wix
- criar colunar auxiliares para tratamentos


checklist:
- Decidir como executar processo de etl
    - qual estrutura de dados vai ser usada 
        - tratamento de dados: duck e dict
        - armazenamento de dados: duck
    - qual Limites de transação de cada api?

- download de posts wix: Completo
    - listar todos posts_id's: completo
    - baixar todos os posts: completo

- converter posts para dict: feito
- coletar versão funcional do main com wallace: feito

- converter post html para rico_documento wix: revisar
    - tags's de titulos
    - tags's de texto
    - tag's de video
    - tag's de decoração
    - tag's de imagens
    - scripts
- 


In [18]:
import json
import pandas as pd

In [22]:
caminho_csv_wordpress = './docs/Posts-Export-2025-July-25-1838.csv'
caminho_posts_wix = './data/posts/'
mapeamento = './data/wix_wordpress_mapping.json'


df = pd.read_csv(caminho_csv_wordpress)
df.head(10)

,ID,check de edição,Title,Content,Excerpt,Date,PostType,Coluna 1,Permalink,ImageURL,...,AuthorLastName,Slug,Format,Template,Parent,ParentSlug,Order,CommentStatus,PingStatus,PostModifiedDate
0,21,True,Débito ou crédito: entenda as vantagens e desv...,"<img class=""size-large wp-image-23 aligncenter...",NaN,2018-02-20,post,NaN,https://otimizeseunegocio.com/saude-financeira...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,debito-ou-credito,NaN,NaN,NaN,NaN,NaN,open,open,2019-02-14
1,30,False,Qual o seu preço? Aprenda este método de calcu...,"<p style=""text-align: center;""><em><strong>Est...",NaN,2018-02-26,post,NaN,https://otimizeseunegocio.com/saude-financeira...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,qual-o-seu-preco-aprenda-este-metodo-de-calcul...,NaN,NaN,NaN,NaN,NaN,open,open,2018-08-08
2,34,False,Qual o seu preço? Parte 2: Aprenda de uma vez ...,"<p style=""text-align: center;""><strong><em>Est...",NaN,2018-02-26,post,NaN,https://otimizeseunegocio.com/saude-financeira...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,qual-o-seu-preco-parte-2-aprenda-de-uma-vez-po...,NaN,NaN,NaN,NaN,NaN,open,open,2018-08-08
3,37,False,5 motivos para NÃO usar um emissor grátis de n...,"<span style=""font-weight: 400;"">Uma das obriga...",NaN,2018-02-26,post,NaN,https://otimizeseunegocio.com/gestao-organizac...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,5-motivos-para-nao-usar-um-emissor-gratis-de-n...,NaN,NaN,NaN,NaN,NaN,open,open,2018-04-13
4,40,False,5 motivos para preferir o certificado A1 (e nã...,"<h2 style=""text-align: center;"">[button type=""...",NaN,2018-02-26,post,NaN,https://otimizeseunegocio.com/legislacao/5-mot...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,5-motivos-para-preferir-o-certificado-a1-e-nao...,NaN,NaN,NaN,NaN,NaN,open,open,2018-07-12
5,47,False,NFCe: posso emitir na minha impressora fiscal?,"<span style=""font-weight: 400;"">O ECF - Emisso...",NaN,2018-02-26,post,NaN,https://otimizeseunegocio.com/legislacao/nfce-...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,nfce-posso-emitir-na-minha-impressora-fiscal,NaN,NaN,NaN,NaN,NaN,open,open,2018-03-13
6,51,False,NF-e e NFC-e: sou obrigado emitir?,"<span style=""font-weight: 400;"">Você abriu sua...",NaN,2018-03-01,post,NaN,https://otimizeseunegocio.com/legislacao/nf-e-...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,nf-e-e-nfc-e-sou-obrigado-emitir,NaN,NaN,NaN,NaN,NaN,open,open,2019-05-10
7,89,False,Como emitir uma nota fiscal eletrônica utiliza...,"<iframe src=""https://www.youtube.com/embed/KdK...",NaN,2018-03-15,post,NaN,https://otimizeseunegocio.com/legislacao/como-...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,como-emitir-uma-nota-fiscal-eletronica-utiliza...,NaN,NaN,NaN,NaN,NaN,open,open,2018-03-17
8,94,False,[Planilha Financeira] Se você cuida bem das su...,"<iframe src=""https://www.youtube.com/embed/ta_...",NaN,2018-03-17,post,NaN,https://otimizeseunegocio.com/gestao-organizac...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,movimento-de-caixa-planilha-gratis,NaN,NaN,NaN,NaN,NaN,open,open,2018-03-17
9,98,False,Como instalar o Emissor Gratuito?,<h4>Antes que você veja sobre o emisssor gratu...,NaN,2018-03-24,post,NaN,https://otimizeseunegocio.com/saude-financeira...,https://otimizeseunegocio.com/wp-content/uploa...,...,Alves,como-instalar-o-emissor-gratuito,NaN,NaN,NaN,NaN,NaN,open,open,2020-09-30


In [21]:
import json
import pandas as pd

# Carregue o arquivo JSON
with open('data/post_exemples/0d274d06-c14e-468e-a2e9-63f7efd57b43.json') as f:
    data = json.load(f)

# Use json_normalize com meta_prefix para evitar conflito de nomes
df_wix = pd.json_normalize(
    data,
    record_path=['richContent', 'nodes'],
    meta=['id', 'title', 'slug', 'firstPublishedDate'],
    meta_prefix='post_'  # Adiciona o prefixo 'post_' às colunas de metadados
)

df_wix.head(10)

,type,id,nodes,imageData.containerData.width.size,imageData.containerData.alignment,imageData.image.src.id,imageData.image.width,imageData.image.height,style.paddingBottom,paragraphData.textStyle.textAlignment,htmlData.containerData.width.custom,htmlData.containerData.alignment,htmlData.html,htmlData.source,post_id,post_title,post_slug,post_firstPublishedDate
0,IMAGE,,[],CONTENT,CENTER,8ef77a_4f32bcc7886348498bf7417df8208578~mv2.jpg,600.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
1,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,2px,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
2,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,NaN,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
3,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,2px,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
4,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,NaN,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
5,HTML,8bab39ef0e5e,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,940px,CENTER,"<div id=""topo-de-funil-92e24d92b708b8251be0""><...",HTML,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
6,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,2px,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
7,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,NaN,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
8,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,2px,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z
9,PARAGRAPH,,"[{'type': 'TEXT', 'id': '', 'nodes': [], 'text...",NaN,NaN,NaN,NaN,NaN,NaN,JUSTIFY,NaN,NaN,NaN,NaN,0d274d06-c14e-468e-a2e9-63f7efd57b43,"Automatizar sua empresa, sim ou não?",automatizar-sua-empresa-sim-ou-não,2025-08-20T23:51:41.148Z


In [3]:
import duckdb

# Connect to an in-memory database
# con = duckdb.connect(database=':memory:', read_only=False)

# Or connect to a file
con = duckdb.connect(database='data/migration.duckdb', read_only=False)

# Create a sample table for demonstration
con.execute("CREATE TABLE items(id INTEGER, name VARCHAR);");
con.execute("INSERT INTO items VALUES (1, 'Laptop'), (2, 'Mouse'), (3, 'Keyboard');");

# Query to show columns from a specific table
table_name = 'items'

# Method 1: Using DESCRIBE
print(f"Columns in '{table_name}' using DESCRIBE:")
describe_result = con.execute(f"DESCRIBE {table_name}").fetchall()
for col in describe_result:
    print(f"- {col[0]} ({col[1]})")

print('\n' + '-'*30 + '\n')

# Method 2: Using PRAGMA table_info
print(f"Columns in '{table_name}' using PRAGMA table_info:")
pragma_result = con.execute(f"PRAGMA table_info('{table_name}')").fetchall()
# The result format is: (column_id, column_name, column_type, nullable, default_value, is_primary_key)
for col in pragma_result:
    print(f"- {col[1]} ({col[2]})")


# Close the connection
con.close()


Columns in 'items' using DESCRIBE:
- id (INTEGER)
- name (VARCHAR)

------------------------------

Columns in 'items' using PRAGMA table_info:
- id (INTEGER)
- name (VARCHAR)


In [ ]:
import duckdb
import json

# 1. Carregar o richContent do arquivo JSON
with open('./data/post_exemples/0e4b70b8-e052-4260-9df2-c2516c6b9685.json') as f:
    post_data = json.load(f)

# Converte o dicionário 'richContent' para uma string JSON
rich_content_string = json.dumps(post_data['richContent'])
post_id = post_data['id']
# Suponha que temos um conteúdo HTML para este post
html_content = "<html><body><h1>Título do Post</h1><p>Este é o conteúdo HTML...</p></body></html>"

# 2. Conectar ao DuckDB e criar a tabela
con = duckdb.connect(database=':memory:') # Usando banco de dados em memória para o exemplo

con.execute("""
CREATE TABLE wix_posts (
    id VARCHAR PRIMARY KEY,
    html_content TEXT,
    rich_content JSON
);
""")

# 3. Inserir os dados na tabela
con.execute(
    "INSERT INTO wix_posts (id, html_content, rich_content) VALUES (?, ?, ?)",
    [post_id, html_content, rich_content_string]
)

# 4. Consultar e extrair dados do JSON
# Exemplo: Extrair o tipo do primeiro nó dentro de 'richContent'
result = con.execute("""
SELECT
    id,
    html_content,
    json_extract(rich_content, '$.nodes[0].type') AS first_node_type
FROM wix_posts;
""").fetchone()

print(result)
# Saída esperada: ('0e4b70b8-e052-4260-9df2-c2516c6b9685', '<html>...</html>', 'PARAGRAPH')

An error occurred: Catalog Error: Table with name my_table does not exist!
Did you mean "pg_tables"?

LINE 1: SELECT * FROM "my_table" LIMIT 5
                      ^
